# rename_cover_images

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import IPython
import shutil

import bookcave

Get book meta data to make deciding on cover images easier.

In [ ]:
_, _, _, _, book_ids, books_df, _, _, _ = bookcave.get_data({'text'}, text_input='filename', return_meta=True)

In [ ]:
amazon_kindle_path = os.path.join('..', 'content', 'amazon_kindle')
folders = os.listdir(amazon_kindle_path)
len(folders)

Rename strangely-named images to `cover.jpg`.

In [ ]:
def start_rename(text_required=False, alt=None, jpg_only=True):
    last_folder = None
    for folder in folders:
        folder_path = os.path.join(amazon_kindle_path, folder)
        
        # Check if 'cover.jpg' already exists.
        cover_path = os.path.join(folder_path, 'cover.jpg')
        if os.path.exists(cover_path):
            continue
        
        # Conditionally check if 'text.txt' exists.
        if text_required and not os.path.exists(os.path.join(folder_path, 'text.txt')):
            continue
        
        # Conditionally check if the alternate cover image exists.
        if alt is not None:
            alt_path = os.path.join(folder_path, alt)
            if os.path.exists(alt_path):
                shutil.copy2(alt_path, cover_path)
                continue

        # Get images in the book folder.
        image_path_sizes = []
        fnames = os.listdir(folder_path)
        for fname in fnames:
            if not fname.endswith('.jpg'):
                continue
            path = os.path.join(folder_path, fname)
            size = os.path.getsize(path)
            image_path_sizes.append((path, size))
        
        # Skip books with no images.
        if len(image_path_sizes) == 0:
            continue
            
        # Sort images in descending order.
        image_path_sizes.sort(key=lambda path_size: path_size[1], reverse=True)

        # Get meta data for this book.
        book_rows = books_df[books_df['asin'].str.match(folder)]
        
        for i, (path, size) in enumerate(image_path_sizes):
            # Clear any previous output in this cell.
            IPython.display.clear_output(wait=True)
            
            # Print last folder.
            if last_folder is not None:
                print('last folder={}\n'.format(last_folder))

            # Display the candidate image.
            image = Image.open(path)
            plt.imshow(image)
            plt.show()
            
            # Ask if it is a cover image.
            for _, book_row in book_rows.iterrows():
                print('Title: {}'.format(book_row['title']))
                print('Authors: {}'.format(book_row['authors']))
            print('Path: {} ({:d} of {:d})'.format(path, i + 1, len(image_path_sizes)))
            print('Size: {}'.format(image.size))
            print('Bytes: {:d}'.format(size))
            print('Cover? [y]/n')
            answer = input()
            if answer == '' or answer == 'y' or answer =='Y':
                shutil.copy2(path, cover_path)
                break
        
        # Keep track of last folder, in case a mistake was made.
        last_folder = folder

In [ ]:
start_rename(text_required=True, alt='0.jpg')